# Geodata.cutout updates

### Jeffrey Feng, 8/16/2021

A short guide to use the product of mask module for masking cutout objects.

## 1. Introduction

Geodata is able to process geospatial data to extract cutouts over specified geographies. Built off the [rasterio library](https://rasterio.readthedocs.io/en/latest/quickstart.html), the mask module imports rasters and shapefiles, merges and flattens multiple layers together, and extracts subsetted cutout data from merged masks and shapefiles.

The sample workflow that shows how to create a mask object and save it to the disk can be found in `mask_creation_workflow.md` and [mask_test.ipynb](https://github.com/east-winds/geodata/tree/mask/tests/mask_test.ipynb).

## 2. Setup

To start, import the required dependencies:

In [ ]:
import geodata
import xarray as xr

To launch a logger for detailed debugging, run:

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

We will use a cutout object created from a downloaded dataset. We first download the dataset through `geodata.Dataset()`.

In `get_data()`, if we specify `testing = true`, the program downloads only first file in download list (e.g., first day of month))

In [ ]:
DS_hourly_test_chn = geodata.Dataset(module = "merra2", 
                                 years = slice(2011, 2011),
                                 month = slice(1,1),
                                 weather_data_config = "slv_radiation_hourly")
if DS_hourly_test_chn.prepared == False: 
    DS_hourly_test_chn.get_data(testing = True)
    
DS_hourly_test_chn.trim_variables()

Extract the cutout from the trimmed dataset.

In [ ]:
cutout = geodata.Cutout(name = "china-2011-slv-hourly-test",
                        module = "merra2",
                        weather_data_config = "slv_radiation_hourly",
                        xs = slice(73, 136), 
                        ys = slice(18, 54), 
                        years = slice(2011, 2011), 
                        months = slice(1,1))
cutout.prepare()

In [ ]:
cutout.meta.time

## 3. Loading mask

**NOTE:** 

If you have created the mask `china_bin`, please ignore the block below, and run the next section. Otherwise, un-comment the cell below and run it.

In [ ]:
# prov_path = shpreader.natural_earth(resolution='10m', category='cultural', name = 'admin_1_states_provinces')
# china_all_shapes = geodata.mask.get_shape(prov_path, key = 'name_en', return_dict = True,
#                          condition_key = 'admin', condition_value = 'China')
# # #get rid of the islands, EDGE CASE cuz the islands are outside of shape
# china_all_shapes.pop(None) 

# china_bin = geodata.Mask("china_bin")
# china_bin.add_layer('FINAL_GRID_5BINS.tif', layer_name = 'bins')

# #extracted shape on bins layer
# china_bin.extract_shapes(china_all_shapes, layer = 'bins')
# china_bin.save_mask()

If the user have already created such mask object on disk, it is sufficient to run the following cell to retrieve the object:

In [ ]:
china_bin = geodata.mask.load_mask("china_bin")

## 4. Adding Mask variables to the Cutout object

### 4.1 Adding masking variables


`add_mask` method will add attribute `merged_mask` and `shape_mask` from the Mask object to the Cutout object. However, the `merged_mask` or `shape_mask` in the Cutout object will be stored in the format of xarray.DataArray, and their dimensions will be coarsened to the same dimension with the Cutout metadata.

The mask `china_bin` has no `merged_mask` value, but the `add_mask` method will look for both `merged_mask` and `shape_mask` attribute saved for the loaded mask, unless the user set the parameter `merged_mask = False`, or `shape_mask = False`


In [ ]:
cutout.add_mask("china_bin")

### 4.2 Adding area variable


The user is also able to add grid area for each grid cell in the cutout metadata. Because the grid cell with the same latitude difference will have different area due to the cylindrical map projection, this method makes sure that the user can capture the variation of grid cell area in the dataset.

In [ ]:
cutout.add_grid_area()

### 4.3 Creating PV data through cutout conversion

The code block below will use the `geodata.convert.pv` method to generate `ds_cutout`, an xarray Dataset that contains the pv variable for the cutout.

`ds_cutout.coarsen(time = 24, boundary = 'exact').mean()` will aggregate the values over its 24 timestamps, since we have pv values for each of the 24 hours in the dataset.

In [ ]:
ds_solar = geodata.convert.pv(cutout, panel = "KANENA", orientation = "latitude_optimal")
ds_solar = ds_solar.reset_coords(['lon', 'lat'], drop = True)
ds_solar = ds_solar.rename({'x': 'lon', 'y': 'lat'})
ds_cutout = ds_solar.to_dataset(name = 'solar')
ds_cutout = ds_cutout.coarsen(time = 24, boundary = 'exact').mean()
ds_cutout = ds_cutout.transpose("time", "lat", "lon")

### 4.4 Combining variables

The `mask` method will mask converted dataSet variable, such as `ds_cutout` created above, with previously added masks and area variable, and return a dictionary of xarray Dataset. Each key in the dictionary is one unique mask from either the merged_mask or shape_mask variable from the Cutout object, and each value is an xarray dataset containing the dataSet variable (`ds_cutout`) with the mask and area values.

The program will automatically search for `merged_mask` and `shape_mask`, unless the user specify `merged_mask = False` or `shape_mask = False`, the masks in `shape_mask` will have the same key as it has in the `shape_mask` attribute, and the mask for `merged_mask` will have the key name "merged_mask".

In [ ]:
combine = cutout.mask(dataset = ds_cutout)
combine.keys()

In [ ]:
combine['Anhui']

In [ ]:
combine['Anhui']['solar'].plot()

In [ ]:
combine['Anhui']['mask'].plot()

In [ ]:
combine['Anhui']['area'].plot()